<a href="https://colab.research.google.com/github/Dedo488/Website/blob/master/Copia_de_template_miniproyecto_1_hive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini-Tarea 1: Análisis de datos de preferencias musicales

La presente actividad debe ser realizada de forma individual. El formato de entregar es el archivo .ipynb con todas las celdas ejecutadas. Las secciones donde se planteen preguntas de forma explícita, deben ser respondida en celdas de texto, y no se aceptará solo el output de una celda de código como respuesta.

**Fecha de entrega:** martes 13 de octubre de 2020, 09:00 hrs.

**Nombre alumno:** COMPLETAR


# Apache Hive

## Instalación de ambiente

Subimos el archivo Hadoop y lo ejecutamos para montar nuestro sistema distribuido en la VM.

In [ ]:
#Subir hadoop_colab_installer.py
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving hadoop_colab_installer.py to hadoop_colab_installer.py
User uploaded file "hadoop_colab_installer.py" with length 13493 bytes


In [ ]:
exec(open('hadoop_colab_installer.py').read())

Active services:
2177 JobHistoryServer
1797 DataNode
2102 NodeManager
2218 Jps
1644 ResourceManager
1727 NameNode



## Descarga de dataset

Descargamos el dataset y revisamos que la información esté en nuestro directorio en la VM.

In [ ]:
!wget -q --show-progress http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-1K.tar.gz
!tar xzf lastfm-dataset-1K.tar.gz

lastfm-dataset-1K.t 100%[===================>] 641.58M  1.73MB/s    in 6m 47s  


In [ ]:
!ls

accumulo		       lastfm-dataset-1K
apache-hive-2.3.7-bin	       lastfm-dataset-1K.tar.gz
apache-hive-2.3.7-bin.tar.gz   mysql-connector-java-8.0.20
apache-hive-bin		       mysql-connector-java-8.0.20.tar.gz
db-derby-10.14.2.0-bin	       pig
db-derby-10.14.2.0-bin.tar.gz  pig-0.17.0
db-derby-bin		       pig-0.17.0.tar.gz
derby.log		       pig_util.py
hadoop			       sample_data
hadoop-2.10.0		       sqoop-1.4.7.bin__hadoop-2.6.0
hadoop-2.10.0.tar.gz	       sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz
hadoop_colab_installer.py      sqoop-bin
hbase			       zookeeper
hcat


In [ ]:
# !cp "/content/drive/My Drive/Colab/lastfm-dataset-1K.tar.gz"/content/

## Actividad 0

Primero revisamos los archivos dentro del directorio lastfm, para conocer los nombres para leerlos. 

Posteriormente creamos un directorio para cada archivo y realizamos el comando -put para colocar nuestros archivos en HDFS. 

Después creamos un archivo .sql con los comandos para crear las tablas de ambos archivos y lo ejecutamos con el comando -f. Con esto, tenemos nuestras tablas en hdfs. 

Por último, ya que anteriormente se borraron los archivos de los directorios, los volvemos a integrar con el comando -put


In [ ]:
!ls lastfm-dataset-1K

README.txt	    userid-timestamp-artid-artname-traid-traname.tsv
userid-profile.tsv


In [ ]:
!hdfs dfs -mkdir userid-profile
!hdfs dfs -mkdir userid-otros
!hdfs dfs -put lastfm-dataset-1K/userid-profile.tsv userid-profile
!hdfs dfs -put lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv userid-otros

In [ ]:
!hdfs dfs -ls userid-profile userid-otros

Found 1 items
-rw-r--r--   1 root supergroup      37842 2020-10-10 21:47 userid-profile/userid-profile.tsv
Found 1 items
-rw-r--r--   1 root supergroup 2529193595 2020-10-10 21:48 userid-timestamp-artid-artname-traid-traname/userid-timestamp-artid-artname-traid-traname.tsv


In [ ]:
%%writefile create_tables.sql

DROP TABLE IF EXISTS user_id_profile;

DROP TABLE IF EXISTS user_id_otros;

CREATE TABLE user_id_profile (userid STRING, gender STRING, age INT, country STRING, signup STRING) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t';
LOAD DATA INPATH 'userid-profile' INTO TABLE user_id_profile;

CREATE TABLE user_id_otros (userid STRING, timeztamp STRING, artist_id STRING, artist_name STRING, track_id STRING, track_name STRING) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t';
LOAD DATA INPATH 'userid-otros' INTO TABLE user_id_otros;


Overwriting create_tables.sql


In [ ]:
!hive -f create_tables.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.0/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.7-bin/lib/hive-common-2.3.7.jar!/hive-log4j2.properties Async: true
OK
Time taken: 8.446 seconds
OK
Time taken: 0.267 seconds
OK
Time taken: 0.928 seconds
Loading data to table default.user_id_profile
OK
Time taken: 0.991 seconds
OK
Time taken: 0.276 seconds
Loading data to table default.user_id_otros
OK
Time taken: 0.553 seconds


In [ ]:
# Ya que se borraron haciendo lo anterior
!hdfs dfs -put lastfm-dataset-1K/userid-profile.tsv userid-profile
!hdfs dfs -put lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv userid-otros

In [ ]:
# Inserte su código para copia de datos en HDFS y creación de tablas en metastore

## Actividad 1

Para la Actividad 1, escribimos la consulta Hivesql para obtener lo necesario. Donde lo primorial es ordernar en forma descendente y colocar un limit 10.

Para este caso la tabla es la siguiente: Donde el más popular para este dataset es Radiohead con 115209 y el último del top ten es Elliott Smith con 50278. 

t2.artist_name	conteo

Radiohead	115209

The Beatles	100338

Nine Inch Nails	84421

Muse	63351

Coldplay	62251

Depeche Mode	59910

Pink Floyd	58561

Death Cab For Cutie	58083

Placebo	53543

Elliott Smith	50278



In [ ]:
%%writefile actividad1.sql
SELECT t2.artist_name, count(t2.userid) as conteo
from user_id_otros as t2
group by artist_name
order by conteo desc
LIMIT 10

Overwriting actividad1_A.sql


In [ ]:
!hive -f actividad1.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.0/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.7-bin/lib/hive-common-2.3.7.jar!/hive-log4j2.properties Async: true
Query ID = root_20201011135414_b2e1ded0-41bc-48ad-b13a-7f4251597bf7
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 10
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.

Responda la siguiente pregunta:

¿ Cuál es el artista más popular y el menos popular dele ranking top-10 ?

## Actividad 2

Para la actividad 2, escribimos la consulta en Hivesql, donde lo primordial es indicar que gender no sea nulo o un string vacío. 

El resultado es la siguiente tabla, donde 63784 hombres y 43748 mujeres han escuchado alguna canción de Radiohead.

t1.gender conteo

m	63784

f	43748

In [ ]:
%%writefile actividad2.sql
SELECT t1.gender, count(*) as conteo
FROM user_id_profile as t1
INNER JOIN user_id_otros as t2
USING (userid)
WHERE t2.artist_name = 'Radiohead' 
and t1.gender is not NULL
and t1.gender <> " "
and t1.gender <> ""
group by t1.gender 
order by conteo desc
LIMIT 10

Overwriting actividad2.sql


In [ ]:
!hive -f actividad2.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.0/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.7-bin/lib/hive-common-2.3.7.jar!/hive-log4j2.properties Async: true
Query ID = root_20201011142319_fff1de67-32b9-40a3-a407-ed95e52ec9f6
Total jobs = 2
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.0/share/hadoop/common/lib/sl

Responda la siguiente pregunta:

¿ Cuál es la cantidad de hombres y mujeres que han escuchado alguna canción del artista más popular ?


## Actividad 3

En la Actividad 3, escribimos la consulta en hivesql y la ordenamos por la edad, la cual es la única variable INT que tenemos. 

age conteo

3	2

4	14

7	2

15	2

16	122

17	444

18	65

19	4514

20	5760

21	1589

22	7038

23	2552

24	1313

25	1289

26	2527

27	2360

28	1330

29	359

30	617

31	146

32	433

33	1441

34	404

35	1543

36	1962

38	422

39	213

42	13

48	35

75	109

103	1

Pero, la edad varía desde 3 a 103 años y 1543 personas de 35 años han escuchado al artista más popular = Radiohead. También podemos notar que este artista es más popular entre las edades [19 a 22]. 

In [ ]:
%%writefile actividad3.sql
SELECT t1.age, count(*) as conteo
FROM user_id_profile as t1
INNER JOIN user_id_otros as t2
USING (userid)
WHERE t2.artist_name = 'Radiohead' 
and t1.age is not null
group by t1.age 
order by t1.age asc

Overwriting actividad3.sql


In [ ]:
!hive -f actividad3.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.0/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.7-bin/lib/hive-common-2.3.7.jar!/hive-log4j2.properties Async: true
Query ID = root_20201011143132_810c98d1-403f-46ec-a6fb-bea4928d0dc6
Total jobs = 2
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.0/share/hadoop/common/lib/sl

Responda la siguiente pregunta:

¿ Cuántos usuarios de 35 años han escuchado al artista más popular ?